<a href="https://www.kaggle.com/code/lazuardialmuzaki/tabu-search-for-hub-allocation-problem?scriptVersionId=143560100" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:,}'.format

## Load all datasets
Comprises of 5 different dataset that encompasses different number of node. 
First data includes 10 nodes, then 25, 55, 81, and 100 respectively.
All of each dataset may consist different matrix of demand flow and matrix of cost.


In [ ]:
flow = pd.read_excel('/kaggle/input/nodes-datasets/CAB10.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/CAB10.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix = np.array(flow)
cost_matrix = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/CAB25.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/CAB25.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_25 = np.array(flow)
cost_matrix_25 = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/TR55.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/TR55.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_55 = np.array(flow)
cost_matrix_55 = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/TR81.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/TR81.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_81 = np.array(flow)
cost_matrix_81 = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/RGP100.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/RGP100.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_100 = np.array(flow)
cost_matrix_100 = np.array(cost)

In [ ]:
flow_matrix

In [ ]:
cost_matrix

### Network Cost

Interhub flow is entitled to discounted rates, defined as following formula:

In [ ]:
def flow_loc(interhub_flow):
    flow_discounted=0
    if interhub_flow < 50000:
        alpha = 1
        intercept = 0
    elif interhub_flow < 100000:
        alpha = 0.8
        intercept = 10000
    elif interhub_flow < 200000:
        alpha = 0.6
        intercept = 30000
    elif interhub_flow >= 200000:
        alpha = 0.4
        intercept = 70000
    else:
        raise ValueError('Invalid interhub flow')
    return alpha, intercept

For flows which are not interhub (for example non-hub node to hub, and hub to non-hub node), the cost is defined as follows:

In [ ]:
def distribution_collection_cost(solution,flow_matrix,cost_matrix):
    total_distribution_cost=0
    for i in range(len(solution)):
        for j in range(len(solution)):
            cost=flow_matrix[i][j]*(cost_matrix[i][solution[i]]+cost_matrix[solution[j]][j])
            total_distribution_cost+=cost
    return total_distribution_cost




As accumulated interhub flow and its corresponding discount has been calculated previously, below code shows the calculation for its cost:

In [ ]:
def interhub_cost(solution,flow_matrix,cost_matrix):
    total_inter_hub_cost=0
    hub_indices =set(solution)
    for hub1 in hub_indices:
        for hub2 in hub_indices:
            if hub1!=hub2:
                inter_hub_flow=0
                for i in range(len(solution)):
                    for j in range(len(solution)):
                        if solution[i]==hub1 and solution[j]==hub2:
                            inter_hub_flow+=flow_matrix[i][j]
                alpha, intercept = flow_loc(inter_hub_flow)
                interhub_cost = cost_matrix[hub1][hub2]*(alpha*inter_hub_flow + intercept)
                total_inter_hub_cost+=interhub_cost
                
    return total_inter_hub_cost


Total cost of interhub and non interhub (distribution and collection)

In [ ]:
def total_cost(solution,flow_matrix,cost_matrix):
    total_cost = distribution_collection_cost(solution,flow_matrix,cost_matrix) + interhub_cost(solution,flow_matrix,cost_matrix)
    return total_cost
    

**Checking Developed Code**

The code is tested using the optimal solution of node array in code below, the result shows that the optimal value is found, denoting that the code for computing hub cost is correct

In [ ]:
total_cost([3,8,8,3,3,8,6,3,8,6], flow_matrix, cost_matrix)

## Initialize Random Solution

In [ ]:
def random_solution(n_hub, n_node):
    solution = [0] * n_node
    hubs = np.random.choice(n_node, n_hub, replace=False)
    
    for item in range(len(solution)):
        for hub in hubs:
            if item == hub:
                solution[item] = hub
                
    for item in range(len(solution)):
        if item not in set(hubs):
            solution[item] = int(np.random.choice(hubs))
    
    return solution

### Neighborhood Structure 1:
Swap the allocation for nodes to different hub

In [ ]:
def NS_1(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    current_solution = current_solution.copy()
#     print('1')
#     counter = 0

    for node in range(len(temp_solution)):
        if node in hub_indices or node in tabu_key:
            continue
        for current_hub in hub_indices:
            for new_hub in hub_indices:
                if current_hub == new_hub:
                    continue
                
                if temp_solution[node] == current_hub:
                    temp_solution[node] = new_hub
                
                    temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
#                     print(temp_solution)
    #                 print(temp_cost)
                    if temp_cost < best_cost:
                        best_solution = temp_solution.copy()
                        best_cost = temp_cost
                        become_tabu1 = node
                    temp_solution = current_solution.copy()
#                     counter +=1
                    
#     print(counter)            
    return best_solution, best_cost, become_tabu1, become_tabu2
        

### Neighborhood Structure 2:

swapping between two hub

In [ ]:
def NS_2(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    
#     print('2')  
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    current_solution = current_solution.copy()
#     counter = 0
    
    
  
    for hub1 in hub_indices:
        for hub2 in hub_indices:
            if hub1 == hub2:
                continue
            if hub1 in tabu_key or hub2 in tabu_key:
                continue
            
#             print(hub1, hub2)
            for node in range(len(temp_solution)):
                if node == hub1 or node == hub2:
                    continue
                if temp_solution[node] == hub1:
                    temp_solution[node] = hub2
                elif temp_solution[node] == hub2:
                    temp_solution[node] = hub1
                    
                if temp_solution == current_solution:
                    continue
                
#             print(temp_solution)
            temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
                

            if temp_cost < best_cost:
                best_solution = temp_solution.copy()
                best_cost = temp_cost
                become_tabu1 = hub1
                become_tabu2 = hub2

            temp_solution = current_solution.copy()

    return best_solution, best_cost, become_tabu1, become_tabu2

In [ ]:
def check_NS_2(hub_indices, tabu_key):
    count = 0
    for item in hub_indices:
        if item in tabu_key:
            count += 1
            if count >= 2:
                return False
    return True

### Neighborhood Structure 3
Swapping spoke to hub, and the other way around

In [ ]:
def NS_3(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    
#     print('3')
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    current_solution = current_solution.copy()
#     counter = 0
    
    for node in range(len(temp_solution)): 
        if node in hub_indices or node in tabu_key:
                continue
        
        for hub in hub_indices:
            
            if hub in tabu_key:
                continue
                
            temp_solution[node] = node
            for other_node in range(len(temp_solution)):
                if other_node == node:
                    continue
                if temp_solution[other_node] == hub:
                    temp_solution[other_node] = node
                    
            temp_solution[hub] = node        
            temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
#             print('node '+str(node))
#             print('hub '+str(hub))
#             print(temp_solution)

            if temp_cost < best_cost:
                best_solution = temp_solution.copy()
                best_cost = temp_cost
                become_tabu1 = node
                become_tabu2 = hub
            temp_solution = current_solution.copy()
#             counter += 1
#     print(counter)
    return best_solution, best_cost, become_tabu1, become_tabu2

### Neighborhood Structure 4
Swapping spoke to hub, and the other way around only for interconnecting hub and spoke

In [ ]:
def NS_4(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    
#     print('4')
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    
    current_solution = current_solution.copy()
#     counter = 0
    
    for node in range(len(temp_solution)): 
        if node in hub_indices or node in tabu_key:
                continue
        
        for hub in hub_indices:
                        
            if temp_solution[node] != hub or hub in tabu_key:
                continue
            
            temp_solution[node] = node
            for other_node in range(len(temp_solution)):
                if other_node == node:
                    continue
                if temp_solution[other_node] == hub:
                    temp_solution[other_node] = node
                    
            temp_solution[hub] = node        
            temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
#             print('node '+str(node))
#             print('hub '+str(hub))
            
#             print(temp_solution)
            if temp_cost < best_cost:
                best_solution = temp_solution.copy()
                best_cost = temp_cost
                become_tabu1 = node
                become_tabu2 = hub
            temp_solution = current_solution.copy()
#             counter += 1
#     print(counter)
    return best_solution, best_cost, become_tabu1, become_tabu2

In [ ]:
NS_set = [NS_1, NS_2, NS_3, NS_4]



## Main Tabu Search
The main function to run the tabu search by inputting parameters, the result will show best solution of node array configuration (which nodes are hub and to which hub each of other nodes connected) and the corresponding best cost.

In [ ]:
def tabu_search(flow_matrix, cost_matrix, n_hub, n_node, tabu_tenure, termination_time):

    current_solution = random_solution(n_hub, n_node)

    hub_indices = set(current_solution)
    current_cost = total_cost(current_solution,flow_matrix,cost_matrix)
    tabu_list = {}
    tabu_key = []
    best_iteration = 0
#     counter = 0
    time_stamp = []
#     print(current_solution)
    start_time = time.time()
    current_time = start_time
    updated = 0
    iteration = 0
    

    final_best_solution = None
    final_best_cost = float('inf')

    temp_solution = current_solution.copy()

    # Initialize list to store best cost for each iteration
    best_cost_list = []
    final_best_cost_list = []
    
    # Initialize list to store best solution for each iteration
    best_solution_list = []
    final_best_solution_list = []

    while current_time - start_time < termination_time:

        if bool(tabu_list):
            keys_to_delete = []
            for key, value in tabu_list.items():
                tabu_list[key] = value - 1
                if tabu_list[key] <= 0:
                    keys_to_delete.append(key)
            for key in keys_to_delete:
                del tabu_list[key]

        tabu_key = list(tabu_list.keys())
                     
        best_solution = None
        become_tabu1 = None
        become_tabu2 = None
        best_cost = float('inf')

        while best_solution == None:
            best_solution, best_cost, become_tabu1, become_tabu2 = np.random.choice(NS_set)(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key)

        best_cost_list.append(best_cost)
        best_solution_list.append(best_solution.copy())

        if best_cost < final_best_cost:
            final_best_solution = best_solution
            final_best_cost = best_cost
            best_iteration = iteration + 1
            updated += 1

        final_best_cost_list.append(total_cost(final_best_solution,flow_matrix,cost_matrix))
        final_best_solution_list.append(final_best_solution.copy())    
            
        tabu_list[become_tabu1] = tabu_tenure
        if become_tabu2 != None:
            tabu_list[become_tabu2] = tabu_tenure

        current_solution = best_solution.copy()
        temp_solution = current_solution.copy()
        hub_indices = set(temp_solution)
        current_time = time.time()
        time_stamp.append(current_time - start_time)
        iteration += 1

    # Plot the two graphs side-by-side in subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].plot(range(iteration), [total_cost(best_solution, flow_matrix, cost_matrix) for best_solution in best_solution_list])
    axs[0].set_xlabel('Iteration')
    axs[0].set_ylabel('Total Cost (Best Solution)')
    axs[0].set_title('Best Solution')
    
    axs[1].plot(range(iteration),[total_cost(best_solution, flow_matrix, cost_matrix) for best_solution in final_best_solution_list])
    axs[1].set_xlabel('Iteration')
    axs[1].set_ylabel('Total Cost (Final Best Solution)')
    axs[1].set_title('Final Best Solution')
    
    plt.show()
    
    return final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp

## RGP100 10 HUB

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_100, cost_matrix_100, 10, 100, 35, 500)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_RGP100_10Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
data = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
data

## RGP100 Hub 7

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_100, cost_matrix_100, 7, 100, 35, 400)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_RGP100_7Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
data = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
data

## TR81 7 Hub

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_81, cost_matrix_81, 5, 81, 25, 140)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_TR81_7Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
data = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
data

## TR81, 5 HUB

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_81, cost_matrix_81, 5, 81, 25, 120)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_TR81_5Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
data = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
data

## TR55, 5 HUB

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_55, cost_matrix_55, 5, 55, 15, 50)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_TR55_5Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
TS_CAB25_3h = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
TS_CAB25_3h

## TR55, 3 HUB

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_55, cost_matrix_55, 3, 55, 15, 35)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_TR55_3Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
data = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
data

## CAB25, 5 HUB

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_25, cost_matrix_25, 5, 25, 8, 10)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_CAB25_5Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
TS_CAB25_3h = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
TS_CAB25_3h

## CAB25, 3 HUB

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = tabu_search(flow_matrix_25, cost_matrix_25, 3, 25, 8, 6)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('Tabu_Search_CAB25_3Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Sheet1')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
TS_CAB25_3h = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
TS_CAB25_3h